In [1]:
!pip install pytorch-lightning

In [2]:
# imports
import os

import torch
from pytorch_lightning import LightningModule, Trainer
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchmetrics import Accuracy
from torchvision import transforms
from torchvision.datasets import MNIST

In [3]:
!git clone https://github.com/vmistry-repo/AI-Novice.git

fatal: destination path 'AI-Novice' already exists and is not an empty directory.


In [4]:
cd /content/AI-Novice/lightning

/content/AI-Novice/lightning


In [5]:
!pip install -r requirements.txt
import main
from models import model
from data import dataset
from utils import utils

CUDA Available:  True


In [6]:
PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 256 if AVAIL_GPUS else 64
AVAIL_GPUS

1

In [7]:
## To get the std and mean value

import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Define the transform to normalize the data
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Load the CIFAR-10 dataset
cifar10_train = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
pixel_means = cifar10_train.data.mean(axis=(0,1,2)) / 255.0
pixel_stds = cifar10_train.data.std(axis=(0,1,2)) / 255.0

print('CIFAR-10 pixel means:', pixel_means)
print('CIFAR-10 pixel stds:', pixel_stds)
print(cifar10_train.data.shape)

Files already downloaded and verified
CIFAR-10 pixel means: [0.49139968 0.48215841 0.44653091]
CIFAR-10 pixel stds: [0.24703223 0.24348513 0.26158784]
(50000, 32, 32, 3)


In [8]:
from trainers import transform as trans

train_transforms = trans.get_train_transforms(pixel_means, pixel_stds)
test_transforms = trans.get_test_transforms(pixel_means, pixel_stds)

trainset = dataset.Cifar10SearchDataset(root='./data', train=True,
                                download=True, transform=train_transforms)
trainset, valset = random_split(trainset, [45000, 5000])
train_loader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(valset, batch_size=BATCH_SIZE, shuffle=True)

testset = dataset.Cifar10SearchDataset(root='./data', train=False,
                               download=True, transform=test_transforms)
test_loader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=True,)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10

class AlbumentationsTransform(pl.LightningDataModule):
    def __init__(self, train_transforms, test_transforms):
        super().__init__()
        self.train_transforms = train_transforms
        self.test_transforms = test_transforms

    def train_transform(self, image):
        return self.train_transforms(image=image)["image"]

    def test_transform(self, image):
        return self.test_transforms(image=image)["image"]

    def train_dataloader(self):
        return train_loader

    def val_dataloader(self):
        return val_loader

    def test_dataloader(self):
        return test_loader

In [10]:
import main
from models import model
from data import dataset
from utils import utils
from trainers import transform

class CustomResnet(LightningModule):
    def __init__(self, data_dir=PATH_DATASETS, hidden_size=16, learning_rate=2e-4, dropout_value=0.05):
        super().__init__()

        # Set our init args as class attributes
        self.data_dir = data_dir
        self.hidden_size = hidden_size
        self.learning_rate = learning_rate
        self.dropout_rate = dropout_value

        # Hardcode some dataset specific attributes
        self.num_classes = 10
        self.transform = test_transforms


        # PrepLayer
        self.preplayer    = self.conv_block(in_channels=3, out_channels=64, kernel_size=(3,3), stride=1, padding=1)
        # Layer 1
        self.conv1_layer1 = self.conv_block(in_channels=64, out_channels=128, kernel_size=(3,3),
                                            stride=1, padding=1, max_pool=True)
        self.resblock1    = self.res_block(in_channels=128, out_channels=128, kernel_size=(3,3))
        # Layer 2
        self.conv2_layer2 = self.conv_block(in_channels=128, out_channels=256, kernel_size=(3,3),
                                            stride=1, padding=1, max_pool=True)
        # Layer 3
        self.conv3_layer3 = self.conv_block(in_channels=256, out_channels=512, kernel_size=(3,3),
                                            stride=1, padding=1, max_pool=True)
        self.resblock2    = self.res_block(in_channels=512, out_channels=512, kernel_size=(3,3))
        # Max Pool
        self.maxpool      = nn.MaxPool2d(kernel_size=(4,4))
        # FC Layer
        self.fc           = nn.Linear(512, 10, bias=False)

        # Define PyTorch model
        self.model = nn.Sequential(
            self.preplayer,
            self.conv1_layer1,
            self.resblock1,
            self.conv2_layer2,
            self.conv3_layer3,
            self.resblock2,
            self.maxpool,
            self.fc
        )

        self.accuracy = Accuracy(task="MULTICLASS", num_classes=10)

    def forward(self, x):
        x  = self.preplayer(x)
        x  = self.conv1_layer1(x)
        r1 = self.resblock1(x)
        x  = x + r1
        x  = self.conv2_layer2(x)
        x  = self.conv3_layer3(x)
        r2 = self.resblock2(x)
        x  = x + r2
        x  = self.maxpool(x)
        x  = x.view(-1, 512)
        x  = self.fc(x)

        return F.log_softmax(x, dim=-1)

    def conv_block(self, in_channels, out_channels, kernel_size, stride=1, padding=0,
                   dropout_value=0, groups=1, dilation=1, max_pool=False):
        x1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                       kernel_size=kernel_size, padding=padding, dilation=dilation,
                       groups=groups, bias=False)
        x2 = nn.MaxPool2d(kernel_size=(2,2), stride=2)
        x3 = nn.BatchNorm2d(out_channels)
        x4 = nn.ReLU()
        x5 = nn.Dropout(dropout_value)
        if max_pool == True:
            return nn.Sequential(x1, x2, x3, x4, x5)
        return nn.Sequential(x1, x3, x4, x5)

    def res_block(self, in_channels, out_channels, kernel_size):
        return nn.Sequential(
                self.conv_block(in_channels, out_channels, kernel_size, stride=1, padding=1),
                self.conv_block(in_channels, out_channels, kernel_size, stride=1, padding=1)
        )

    def dial_conv_block(self, in_channels, out_channels, kernel_size, padding, dropout_value, groups=1, dilation=1):
        return nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, padding=padding, dilation=dilation, groups=groups, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(out_channels),
            nn.Dropout(dropout_value),
            nn.Conv2d(in_channels=out_channels, out_channels=in_channels, kernel_size=(1, 1), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(in_channels),
            nn.Dropout(dropout_value),
        )

    def output_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.AvgPool2d(kernel_size=26),
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=(1, 1), padding=0, bias=False)
        )

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.accuracy(preds, y)

        # Calling self.log will surface up scalars for you in TensorBoard
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", self.accuracy, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        # Here we just reuse the validation_step for testing
        return self.validation_step(batch, batch_idx)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

    ####################
    # DATA RELATED HOOKS
    ####################

    def prepare_data(self):
        self.train_data, self.test_data = trainset, testset

    def setup(self, stage=None):
        self.train_data, self.test_data = trainset, testset
        # Assign train/val datasets for use in dataloaders
        if stage == "fit" or stage is None:
            self.cifar_train, self.cifar_train_val = trainset, valset

        # Assign test dataset for use in dataloader(s)
        if stage == "test" or stage is None:
            self.cifar_test = self.test_data

    def train_dataloader(self):
        return train_loader

    def val_dataloader(self):
        return val_loader

    def test_dataloader(self):
        return test_loader

In [11]:
from pytorch_lightning.callbacks import ModelSummary

model = CustomResnet()

trainer = Trainer(
    callbacks=[ModelSummary(max_depth=-1)],
    precision="16-mixed",
#    gpus = AVAIL_GPUS,
    max_epochs = 20,
    accelerator="gpu"
#    progress_bar_refresh_rate=10
)

data_module = AlbumentationsTransform(train_transforms, test_transforms)

trainer.fit(model, datamodule=data_module)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
   | Name           | Type               | Params
-------------------------------------------------------
0  | preplayer      | Sequential         | 1.9 K 
1  | preplayer.0    | Conv2d             | 1.7 K 
2  | preplayer.1    | BatchNorm2

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:480: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [12]:
trainer.test(model, datamodule=data_module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:480: PossibleUserWarning: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │    0.8827999830245972     │
│         val_loss          │    0.3893430829048157     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 0.3893430829048157, 'val_acc': 0.8827999830245972}]